# Searching and Downloading NSIDC Cloud Collections

## Requirements

* [NASA Eartdata Login (EDL) credentials](https://urs.earthdata.nasa.gov/)
* python libraries:
  - aws-cli
  - xarray
  - fs-s3fs
  - zarr
  - [cmr](https://github.com/nasa/eo-metadata-tools/tree/master/CMR/python)
      - [python-cmr](https://github.com/nsidc/python-cmr) (optional fallback) NSIDC fork

## Querying CMR

### Public collections

### Private collections

## Data Access using AWS S3

### Downloading files on S3 using the official aws-cli library

### Using xarray to open files on S3

### Dask
